In [2]:
proj_dir = 'D:/Projects/MIDS/w266/final_project/'

BERT_BASE_DIR = proj_dir + '/bert/models/chinese_L-12_H-768_A-12'
TMP = 'C:/tmp'

In [3]:
import sys
import os
sys.path.append(proj_dir)
os.chdir(proj_dir)

In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

In [5]:
## Split test, train and dev data

In [6]:
## by category
csc_dir = 'data/raw/chnsenticorp/'

category = {'ctrip': 'Ctrip_htl_4000/Ctrip_htl_ba_4000/',
            'dangdang': 'Dangdang_Book_4000/',
            'jingdong': 'Jingdong_NB_4000/'}

label = {'pos': '1', 'neg': '0'}

def load_contents(cat, sent):
    contents = []
    file_dir = csc_dir + category[cat] + sent + '/'
    files = os.listdir(file_dir)
    for fn in files:
        with open(file_dir + fn, 'r') as f:
            try:
                s = f.read()
                s = s.replace('\n', '') + '\t' + label[sent] + '\n'
                contents.append(s)
            except:
                continue
    return contents

In [7]:
ctrip_pos = load_contents('ctrip', 'pos')
ctrip_neg = load_contents('ctrip', 'neg')

dangdang_pos = load_contents('dangdang', 'pos')
dangdang_neg = load_contents('dangdang', 'neg')

jingdong_pos = load_contents('jingdong', 'pos')
jingdong_neg = load_contents('jingdong', 'neg')

In [8]:
def split_data(cat):
    test = []
    train = []
    dev = []
    for sent in ('pos', 'neg'):
        data = eval("{}_{}".format(cat, sent))
        X_dev, X_test = train_test_split(data, test_size=0.2, random_state=1)
        X_train, X_dev = train_test_split(X_dev, test_size=0.2, random_state=1)
        
        test += X_test
        train += X_train
        dev += X_dev
    
    test = shuffle(test, random_state=1)
    train = shuffle(train, random_state=1)
    dev = shuffle(dev, random_state=1)
    
    return test, train, dev

In [9]:
ctrip_test, ctrip_train, ctrip_dev = split_data('ctrip')
dangdang_test, dangdang_train, dangdang_dev = split_data('dangdang')
jingdong_test, jingdong_train, jingdong_dev = split_data('jingdong')

In [10]:
all_test = ctrip_test + dangdang_test + jingdong_test
all_train = ctrip_train + dangdang_train + jingdong_train
all_dev = ctrip_dev + dangdang_dev + jingdong_dev

all_test = shuffle(all_test, random_state=1)
all_train = shuffle(all_train, random_state=1)
all_dev = shuffle(all_dev, random_state=1)

In [14]:
# Write files, choose the category
valid_cats = ['ctrip', 'dangdang', 'jingdong', 'all']

for ftype in ['test', 'train', 'dev']:
    for cat in valid_cats:
        data = eval("{}_{}".format(cat, ftype))
        fn = proj_dir + 'data/processed/csc/{cat}/{ftype}.tsv'.format(cat=cat, ftype=ftype)
        with open(fn, 'w', encoding='utf-8') as f:
            f.writelines(data)